In [1]:
import pandas as pd
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup, AdamW

In [2]:
def categorizer(label):
    
    if label == 'positive':
        return 2
    elif label == 'neutral':
        return 1
    elif label == 'negative':
        return 0



def convert_idx_to_sentiment(idx):
    
    return ['negative', 'neutral', 'positive'][idx]

In [3]:
class FinancialPhraseBankDataset(Dataset):
    
    def __init__(
            self,
            data: pd.DataFrame,
            tokenizer: BertTokenizer,
            text_max_token_length: int = 512
            ):
                
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_length = text_max_token_length
        
    
    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index: int):
        
        data_row = self.data.iloc[index]
    
        encoded_text = self.tokenizer.encode_plus(
            data_row.phrase,
            max_length = self.text_max_token_length, 
            padding = 'max_length',
            truncation = True, 
            return_attention_mask = True, 
            add_special_tokens = True, 
            return_tensors = 'pt'
            )
        
        return dict(
            input_ids = encoded_text.input_ids.flatten(),
            attention_mask = encoded_text.attention_mask.flatten(),
            token_type_ids = encoded_text.token_type_ids.flatten(),
            label = torch.tensor(data_row.sentiment).unsqueeze(0)
            )
    
    
    
class FinancialPhraseBankDataModule(pl.LightningDataModule):
    
    def __init__(            
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: BertTokenizer,
        batch_size: int = 64,
        text_max_token_length: int = 512,
    ):
        
        super().__init__()
        
        self.train_df = train_df
        self.test_df = test_df
        
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_max_token_length = text_max_token_length
        
        self.setup()
        
        
    def __len__(self):
        return len(self.train_df)
        


    def setup(self, stage = None):
        self.train_dataset = FinancialPhraseBankDataset(
            self.train_df,
            self.tokenizer,
            self.text_max_token_length,
            )
        
        self.test_dataset = FinancialPhraseBankDataset(
            self.test_df,
            self.tokenizer,
            self.text_max_token_length,
            )
    
    
    def train_dataloader(self):        
        return DataLoader(
            self.train_dataset,
            batch_size = self.batch_size,
            shuffle = False
            )

    
    def val_dataloader(self):        
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size,
            shuffle = False
            )
    
    
    def test_dataloader(self):        
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size,
            shuffle = False
            )

In [4]:
class FinBERT(pl.LightningModule):
    
    def __init__(self, model_path = 'model', train_samples = 3388, batch_size = 64, epochs = 10, num_labels = 3, learning_rate = 2e-5, discriminative_fine_tuning_rate = 0.85):
        super().__init__()
    
        self.learning_rate = learning_rate
        self.discriminative_fine_tuning_rate = discriminative_fine_tuning_rate
        self.train_samples = train_samples
        self.batch_size = batch_size
        self.gradient_accumulation_steps = 1
        self.epochs = epochs
        self.warm_up_proportion = 0.2
        self.num_train_optimization_steps = int(self.train_samples / self.batch_size / self.gradient_accumulation_steps) * epochs
        self.num_warmup_steps = int(float(self.num_train_optimization_steps) * self.warm_up_proportion)


        self.no_decay_layer_list = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

        config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states = True)
        config.num_labels = num_labels
        self.bert_model = BertForSequenceClassification.from_pretrained(model_path, config = config)
        
        self.optimizer_grouped_parameters = self.get_optimizer_grouped_parameters()
        
        self.criterion = nn.CrossEntropyLoss()
        
        
    def forward(self, input_ids, attention_mask, token_type_ids, labels = None):        
        output = self.bert_model(
            input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            labels = labels
            )
         
        return output.loss, output.logits
    
    
    def get_optimizer_grouped_parameters(self):
        
        discriminative_fine_tuning_encoders = []
        for i in range(12):
            ith_layer = list(self.bert_model.bert.encoder.layer[i].named_parameters())
            
            encoder_decay = {
                'params': [param for name, param in ith_layer if
                           not any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
                'weight_decay': 0.01,
                'lr': self.learning_rate / (self.discriminative_fine_tuning_rate ** (12 - i))
                }
        
            encoder_nodecay = {
                'params': [param for name, param in ith_layer if
                           any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
                'weight_decay': 0.0,
                'lr': self.learning_rate / (self.discriminative_fine_tuning_rate ** (12 - i))}
            
            discriminative_fine_tuning_encoders.append(encoder_decay)
            discriminative_fine_tuning_encoders.append(encoder_nodecay)
            
        
        embedding_layer = self.bert_model.bert.embeddings.named_parameters()
        pooler_layer = self.bert_model.bert.pooler.named_parameters()
        classifier_layer = self.bert_model.classifier.named_parameters()
        
        optimizer_grouped_parameters = [
            {'params': [param for name, param in embedding_layer if
                        not any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
             'weight_decay': 0.01,
             'lr': self.learning_rate / (self.discriminative_fine_tuning_rate ** 13)},
            {'params': [param for name, param in embedding_layer if
                        any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
             'weight_decay': 0.0,
             'lr': self.learning_rate / (self.discriminative_fine_tuning_rate ** 13)},
            {'params': [param for name, param in pooler_layer if
                        not any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
             'weight_decay': 0.01,
             'lr': self.learning_rate},
            {'params': [param for name, param in pooler_layer if
                        any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
             'weight_decay': 0.0,
             'lr': self.learning_rate},
            {'params': [param for name, param in classifier_layer if
                        not any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
             'weight_decay': 0.01,
             'lr': self.learning_rate},
            {'params': [param for name, param in classifier_layer if
                        any(no_decay_layer_name in name for no_decay_layer_name in self.no_decay_layer_list)],
             'weight_decay': 0.0,
             'lr': self.learning_rate}            
            ]
                
        optimizer_grouped_parameters.extend(discriminative_fine_tuning_encoders)
        
        return optimizer_grouped_parameters
    
    
    def training_step(self, batch, batch_index):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        label = batch['label']
        
        loss, logits = self(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            labels = label
            )
        
        total = label.size(0)        
        pred = torch.argmax(logits, 1).unsqueeze(1)
        correct = (pred == label).sum().item()
        acc = correct/total

        
        self.log('train_loss', loss, prog_bar = True, logger = True)
        self.log('train_acc', acc, prog_bar = True, logger = True)
        
        return loss
    
    
    def validation_step(self, batch, batch_index):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        label = batch['label']

        
        loss, logits = self(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            labels = label
            )
        
        total = label.size(0)        
        pred = torch.argmax(logits, 1).unsqueeze(1)
        correct = (pred == label).sum().item()
        acc = correct/total

        self.log('val_acc', acc, prog_bar = True, logger = True)
        self.log('val_loss', loss, prog_bar = True, logger = True)
        
        return loss
    
    
    def test_step(self, batch, batch_index):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']        
        label = batch['label']

        loss, logits = self(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            labels = label
            )
        
        total = label.size(0)        
        pred = torch.argmax(logits, 1).unsqueeze(1)
        correct = (pred == label).sum().item()
        acc = correct/total
        
        self.log('test_acc', acc, prog_bar = True, logger = True)
        self.log('test_loss', loss, prog_bar = True, logger = True)
        
        return loss
    
    
    def configure_optimizers(self):
        
        optimizer = AdamW(
            self.optimizer_grouped_parameters,
            lr = self.learning_rate,
            correct_bias = False
            )

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps = self.num_warmup_steps,
            num_training_steps = self.num_train_optimization_steps
            )
        
        return [optimizer], [{'scheduler': scheduler, 'interval': 'step'}]

In [5]:
financial_phrase_dataset = pd.read_csv('dataset/financial_phrase_bank/all-data.csv', encoding = 'latin-1', names = ['sentiment', 'phrase']).drop_duplicates().dropna().reset_index(drop = True)
financial_phrase_dataset.sentiment = financial_phrase_dataset.sentiment.apply(lambda x: categorizer(x))
train, test = train_test_split(financial_phrase_dataset, test_size = 0.3, shuffle = True)

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

EPOCHS = 10
BATCH_SIZE = 32
NUM_LABELS = 3
LEARNING_RATE = 2e-5    
DISCRIMINATIVE_FINE_TUNING_RATE = 0.85

data_module = FinancialPhraseBankDataModule(train, test, tokenizer, batch_size = BATCH_SIZE)    
model = FinBERT(model_path = 'model', train_samples = len(data_module), batch_size = BATCH_SIZE, epochs = EPOCHS, num_labels = NUM_LABELS, learning_rate = LEARNING_RATE, discriminative_fine_tuning_rate = DISCRIMINATIVE_FINE_TUNING_RATE)

checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename = 'best-checkpoint',
    save_top_k = 1,
    verbose = True,
    monitor = 'val_loss',
    mode = 'min'
    )

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 2,
    mode = 'min'
    )

logger = TensorBoardLogger('lightning_logs', name = 'finbert_sentiment')

Some weights of the model checkpoint at model were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at model and a

In [7]:
trainer = pl.Trainer(
    logger = logger,
    callbacks = [checkpoint_callback, early_stopping],
    max_epochs = EPOCHS,
    gpus = 1,
    progress_bar_refresh_rate = 1
    )

trainer.fit(model, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                          | Params
-------------------------------------------------------------
0 | bert_model | BertForSequenceClassification | 109 M 
1 | criterion  | CrossEntropyLoss              | 0     
-------------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.938   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 105: val_loss reached 0.49629 (best 0.49629), saving model to "/home/ubuntu/Documents/shared/python_code/finbert/checkpoints/best-checkpoint.ckpt" as top 1
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 211: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 2, global step 317: val_loss was not in top 1


In [8]:
def get_sentiment(text):
    
    encoded_text = tokenizer.encode_plus(
        text,
        max_length = 512,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = 'pt'
        )
    
    logit_output = sentiment_model(
        input_ids = encoded_text.input_ids.flatten().unsqueeze(0),
        attention_mask = encoded_text.attention_mask.flatten().unsqueeze(0),
        token_type_ids = encoded_text.token_type_ids.flatten().unsqueeze(0)
        )[-1]
    
    predicted_sentiment = torch.argmax(logit_output, 1)
    
    return convert_idx_to_sentiment(predicted_sentiment)

In [12]:
sentiment_model = FinBERT.load_from_checkpoint('checkpoints/best-checkpoint.ckpt')
sentiment_model.freeze()

Some weights of the model checkpoint at model were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at model and a

In [13]:
get_sentiment('Apple, Facebook and Microsoft Are Still Undervalued')

'positive'

In [14]:
get_sentiment('Apple’s new debt deal could mean more shareholder rewards after blowout earnings')

'positive'

In [15]:
get_sentiment('Apple is delaying its office return by at least a month because of rising COVID-19 cases, report says')

'negative'

In [17]:
get_sentiment('''iPhone 13 release date: Apple's new flagship could be coming in September''')

'neutral'